In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score

/u/amo-d1/grad/sma340/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/amo-d1/grad/sma340/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_training_path = "/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/dialogs/training"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_training_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_training_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_train = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_train)}")

100%|██████████| 8720/8720 [01:10<00:00, 122.88it/s]

8720
8720
8720
8720

number of rows: 8720


In [5]:
df_recommender_train = df_recommender_train[(df_recommender_train["recommend_indexes"] != -1) & (df_recommender_train["turns"].apply(lambda x: len(x) > 0))]

In [6]:
apps_training_path = "/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/data/master_app_data_V1_true.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())

In [7]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_train['recommended_app_name'] = df_recommender_train.apply(fix_recommended_apps_names, axis=1)

In [8]:
cols = ['app_name','app_type','app_category']
df_app = pd.read_csv(apps_training_path, usecols=cols)
df_apps = df_app.map(lambda x: x.lower() if isinstance(x, str) else x)
candidate_apps = []
def filter_candidate_apps(rec_app_name):
    
    df_rec_app=df_apps[df_apps['app_name'] == rec_app_name]
    recommended_app_type = df_rec_app['app_type'].iloc[0]
    recommended_app_category = df_rec_app['app_category'].iloc[0]
    category_filtered = df_apps[df_apps['app_category'] == recommended_app_category]
    df_apps_filter_1 = df_apps[df_apps['app_category'] != recommended_app_category]
    #print(recommended_app_type, recommended_app_category)
    #print(len(category_filtered))
    
    max_ratio = 0
    best_match = None
    for index, row in category_filtered.iterrows():
        if len(candidate_apps) >= 25:
            break
        if fuzz.ratio(row["app_type"], recommended_app_type) > 30:
            app = row['app_name']
            
            candidate_apps.append(app)
    
    for index, row in df_apps_filter_1.iterrows():
        if len(candidate_apps) >= 25:
            break
        ratio = fuzz.ratio(row["app_type"], recommended_app_type)
        if ratio > max_ratio:
            max_ratio = ratio
            app = row['app_name']
            best_match = app
            candidate_apps.append(best_match)
    
    apps_to_remove_2 = set(candidate_apps)
    df_apps_filter_2 = df_apps[~df_apps['app_name'].isin(apps_to_remove_2)]

    for index, row in df_apps_filter_1.iterrows():
        if len(candidate_apps) >= 25:
            break
        app = row['app_name']
        candidate_apps.append(app)

    
     
    return candidate_apps

In [9]:
def candidate_creator(row):
    selected_values = np.random.choice(np.setdiff1d(filter_candidate_apps(row["recommended_app_name"]), [row["recommended_app_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_train['candidate_apps'] = df_recommender_train.apply(candidate_creator, axis=1)

In [10]:
df_recommender_train["recommended_app_name"][874] 

'six pack in 30 days'

In [11]:
df_recommender_train["turns"][874][df_recommender_train["recommend_indexes"][874]]

'computer: alright, then i recommend giving the six pack in 30 days app a try.'

In [12]:
df_recommender_train["candidate_apps"][874]

array(['lemmings', 'six pack in 30 days',
       'empires & puzzles: match-3 rpg', 'my home design - modern city',
       'sweet fruit candy', 'angry birds dream blast', 'human: fall flat',
       'fishdom', 'disney frozen free fall games', 'bad piggies hd',
       'antistress - relaxation toys', 'magic jigsaw puzzles - game hd',
       'drop the number® : merge game', 'bad piggies', 'flow free',
       'lily’s garden', 'candy crush jelly saga', 'dop 4: draw one part',
       'bubble shooter: panda pop!', 'sudoku - classic sudoku puzzle',
       'cut the rope', 'math | riddles and puzzles maths games',
       'sweet escapes: design a bakery with puzzle games',
       'bubble shooter rainbow', 'pokémon café remix'], dtype='<U48')

In [13]:
conversation_validation_path = "/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/dialogs/validation"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_validation_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_validation_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_validation = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_validation)}")

  6%|▋         | 81/1285 [00:00<00:02, 597.35it/s]

100%|██████████| 1285/1285 [00:03<00:00, 346.75it/s]

1285
1285
1285
1285

number of rows: 1285


In [14]:
df_recommender_validation = df_recommender_validation[(df_recommender_validation["recommend_indexes"] != -1) & (df_recommender_validation["turns"].apply(lambda x: len(x) > 0))]

In [15]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_validation['recommended_app_name'] = df_recommender_validation.apply(fix_recommended_apps_names, axis=1)

In [16]:
def candidate_creator(row):
    selected_values = np.random.choice(np.setdiff1d(filter_candidate_apps(row["recommended_app_name"]), [row["recommended_app_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_validation['candidate_apps'] = df_recommender_validation.apply(candidate_creator, axis=1)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:", "candidate_apps:", "previous_interactions:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

In [18]:
prompt_validation = []
recommend_validation = []
for _, row in df_recommender_validation.iterrows():
    if row["previous_interactions"] is not None:
        prompt = "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_validation.append(prompt)
            recommend_validation.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

In [19]:
prompt_train = []
recommend_train = []
for _, row in df_recommender_train.iterrows():
    if row["previous_interactions"] is not None:
        prompt = "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_train.append(prompt)
            recommend_train.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")
            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

In [20]:
recommend_encodings["input_ids"][10]

tensor([24435, 12245,   277,  1120,     5,     1,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])

In [21]:
prompt_train

["previous_interactions: calm - meditate, sleep, relax, kahoot! play & create quizzes\ncomputer: hi! how can i help you today?\nhuman: i'm looking for puzzle game apps for android devices.\ncomputer: what type of puzzle games do you like?\nhuman: i want apps with interactive gameplay.\ncomputer: are you looking for apps with a rating of at least 4.3 stars?\nhuman: yes, that's what i prefer.\ncomputer: do you want a free or paid app?\nhuman: free apps, please.\ncomputer: do you want apps with a minimum of 10m user reviews?\nhuman: yes, i prefer popular apps.\ncomputer: do you have a preference for the content rating?\nhuman: everyone is fine for me.\ncomputer: i recommend trying the where's my water? 2 app based on your interests.\nhuman: how are the ads in that app?\ncomputer: some users have mentioned there are too many ads at times.\nhuman: i think i'll try a different app.\ncomputer: how about trying the toy blast app?\nhuman: can you tell me more about toy blast?\ncomputer: it has 

In [22]:
recommend_train

['fishdom app.',
 'uno!™ app.',
 'sonic dash - endless running app.',
 'pixel gun 3d - battle royale app.',
 'stick war: legacy app.',
 'marvel contest of champions app.',
 'vpn proxy master - safer vpn app.',
 'drum pad machine - beat maker app.',
 'miga town: my world app.',
 'journeys: interactive series app.',
 'subway surfers app.',
 'soundcloud: play music & songs app.',
 'my talking tom friends app.',
 'traffic rider app.',
 'basketball battle app.',
 'faceapp: face editor app.',
 'google pay: save, pay, manage app.',
 'vector app.',
 'tinder - dating & make friends app.',
 'carrom pool: disc game app.',
 'among us app.',
 "evony: the king's return app.",
 'adorable home app.',
 'alibaba.com app.',
 'tubi - movies & tv shows app.',
 'tomb of the mask app.',
 'the grand mafia app.',
 'shopee ph: shop on 3.3 - 3.15 app.',
 'gacha life app.',
 'whatsapp messenger app.',
 'vita - video editor & maker app.',
 'piano fire: edm music & piano app.',
 'off the road - otr open world drivi

In [23]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [25]:
training_args = TrainingArguments(
    output_dir="/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/outputs_T5_with_interactions_with_candidate_apps",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=1000,#self.cfg.save_steps,
    eval_steps=1000, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
trainer.train()
trainer.save_model()

Step,Training Loss,Validation Loss
1000,0.035200,0.032574
2000,0.019300,0.025337


Test the model

In [3]:
torch.cuda.empty_cache()

In [3]:
def is_approximate_substring(substring, string, threshold=70):
    for i in range(len(string) - len(substring) + 1):
        window = string[i:i+len(substring)]
        similarity_ratio = fuzz.ratio(substring, window)
        if similarity_ratio >= threshold:
            return True
    return False

In [4]:
conversation_test_path = "/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/dialogs/testing"

user_id = []
previous_interactions = []
recommended_app_name = []
turns = []
recommend_indexes = []


# List all files in the folder
files = os.listdir(conversation_test_path)

# Loop through each file in the directory
for filename in tqdm(files):
    # Construct the full file path
    file_path = os.path.join(conversation_test_path, filename)

    # Check if the current item is a file
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path, 'r') as file:
            # Read the content of the file
            file_content = file.read().lower()
            
            # finding the user id
            index_1 = file_content.find("user's previous interactions")
            user_id.append(file_content[9:index_1].rstrip('\n'))
            
            # finding the User's Previous Interactions
            index_2 = file_content.find("recommended app name:")
            previous_interactions_arr = file_content[index_1+29:index_2].rstrip('\n').split("app name:")
            previous_interactions_arr_filtered = []
            for previous_interaction in previous_interactions_arr[1:]:
                previous_interactions_arr_filtered.append(previous_interaction[:previous_interaction.find(" | ")])
            if len(previous_interactions_arr_filtered) > 0:
                previous_interactions.append(",".join(previous_interactions_arr_filtered))
            else:
                previous_interactions.append(None)
            
            # finding recommended app name
            index_3 = file_content[index_2:].find("package name")
            recommended = file_content[index_2+22:index_2+index_3-3].rstrip('\n')
            recommended_app_name.append(recommended)
            
            # finding each turns
            dialog_turns = []
            dialog_index = 0
            COMPUTER_index = file_content.find("computer:")
            file_content = file_content[COMPUTER_index:]
            found_recommender = False
            while True:
                HUMAN_index = file_content.find("human:")
                if HUMAN_index == -1:
                    break
                turn = file_content[:HUMAN_index].rstrip('\n') # computer dialog
                if (recommended in turn) and not found_recommender:
                    recommend_indexes.append(dialog_index)
                    found_recommender = True
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[HUMAN_index:]
                
                COMPUTER_index = file_content.find("computer:")
                turn = file_content[:COMPUTER_index].rstrip('\n') # human dialog
                dialog_turns.append(turn)
                dialog_index +=1
                file_content = file_content[COMPUTER_index:]
                
            if not found_recommender: # approximately finding the recommender turn
                for i, dialog_turn in enumerate(dialog_turns):
                    if is_approximate_substring(recommended, dialog_turn):
                        recommend_indexes.append(i)
                        found_recommender = True
                        break
                    
            if not found_recommender:
                recommend_indexes.append(-1)
                        
            turns.append(dialog_turns)

print(len(user_id))
print(len(previous_interactions))
print(len(recommended_app_name))
print(len(recommend_indexes))
df_recommender_test = pd.DataFrame({"user_id": user_id, "previous_interactions":previous_interactions, "recommended_app_name":recommended_app_name, "turns": turns, "recommend_indexes":recommend_indexes})
print(f"\nnumber of rows: {len(df_recommender_test)}")

100%|██████████| 3290/3290 [00:15<00:00, 206.49it/s]

3290
3290
3290
3290

number of rows: 3290


In [5]:
df_recommender_test = df_recommender_test[(df_recommender_test["recommend_indexes"] != -1) & (df_recommender_test["turns"].apply(lambda x: len(x) > 0))]

In [6]:
apps_training_path = "/u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/data/master_app_data_V1_true.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())

In [7]:
def fix_recommended_apps_names(row):
    if row["recommended_app_name"] not in all_apps:
        for app in all_apps:
            if fuzz.ratio(row["recommended_app_name"], app) > 80:
                return app
        return "uno!™"
    else:
        return row["recommended_app_name"]

df_recommender_test['recommended_app_name'] = df_recommender_test.apply(fix_recommended_apps_names, axis=1)

In [ ]:
cols = ['app_name','app_type','app_category']
df_app = pd.read_csv(apps_training_path, usecols=cols)
df_apps = df_app.applymap(lambda x: x.lower() if isinstance(x, str) else x)
candidate_apps = []
def filter_candidate_apps(rec_app_name):
    
    df_rec_app=df_apps[df_apps['app_name'] == rec_app_name]
    recommended_app_type = df_rec_app['app_type'].iloc[0]
    recommended_app_category = df_rec_app['app_category'].iloc[0]
    category_filtered = df_apps[df_apps['app_category'] == recommended_app_category]
    df_apps_filter_1 = df_apps[df_apps['app_category'] != recommended_app_category]
    #print(recommended_app_type, recommended_app_category)
    #print(len(category_filtered))
    
    max_ratio = 0
    best_match = None
    for index, row in category_filtered.iterrows():
        if len(candidate_apps) >= 25:
            break
        if fuzz.ratio(row["app_type"], recommended_app_type) > 30:
            app = row['app_name']
            
            candidate_apps.append(app)
    
    for index, row in df_apps_filter_1.iterrows():
        if len(candidate_apps) >= 25:
            break
        ratio = fuzz.ratio(row["app_type"], recommended_app_type)
        if ratio > max_ratio:
            max_ratio = ratio
            app = row['app_name']
            best_match = app
            candidate_apps.append(best_match)
    
    apps_to_remove_2 = set(candidate_apps)
    df_apps_filter_2 = df_apps[~df_apps['app_name'].isin(apps_to_remove_2)]

    for index, row in df_apps_filter_1.iterrows():
        if len(candidate_apps) >= 25:
            break
        app = row['app_name']
        candidate_apps.append(app)

    
     
    return candidate_apps

In [8]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(filter_candidate_apps(row["recommended_app_name"]), [row["recommended_app_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app_name"]) 

df_recommender_test['candidate_apps'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [9]:
prompt_test = []
recommend_test = []
candidate_apps = []
true_candidate_index = []
for _, row in df_recommender_test.iterrows():
    # creating candidate apps
    candidates = []
    for index, candidate_app in enumerate(row["candidate_apps"].tolist()):
        candidates.append(candidate_app + " app.")
        if candidate_app == row["recommended_app_name"]:
            true_candidate_index.append(index)
    candidate_apps.append(candidates)
    if row["previous_interactions"] is not None:
        prompt = "previous_interactions:" + row["previous_interactions"] + "\n"
    else:
        prompt = "previous_interactions: No previous interactions" + "\n"
    for index, turn in enumerate(row["turns"]):
        if index < row["recommend_indexes"]:
            prompt += turn + "\n"
        elif index == row["recommend_indexes"]:
            prompt += "candidate_apps: "
            for app in row["candidate_apps"]:
                prompt += "'" + app + "', "
            prompt += "\n"
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(row["recommended_app_name"] + " app.")
            break
        else:
            print("error!!")

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "u/amo-d1/grad/sma340/project/llmrank/MobileConvRec-Main/outputs_T5_with_interactions_with_candidate_apps")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:",  "candidate_apps:", "previous_interactions:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

In [11]:
# bleu = evaluate.load("bleu")

In [10]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" app.", "")  for generation in generations_predicted]
    generation_batch = [generation.replace(" app.", "") for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [14]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/410 [00:00<?, ?it/s]

100%|██████████| 410/410 [07:09<00:00,  1.05s/it]

success_rate:  0.5370992366412214


In [11]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [19]:
scores = recommender_rank(prompt_test, candidate_apps, model, tokenizer, batch_size=8)

100%|██████████| 10235/10235 [40:13<00:00,  4.24it/s]


In [24]:
top_k_accuracy_score(true_candidate_index, scores, k=10)

0.8732824427480916

In [25]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_apps[0]))] for index in true_candidate_index]

In [35]:
ndcg_score(true_relevance, scores, k=10)

0.6961009673926019

In [21]:
import pickle

# Open a file in binary write mode
with open('scores_FlanT5_with_AC_PI.pkl', 'wb') as file:
    # Use pickle to dump the array into the file
    pickle.dump(scores, file)

In [14]:
# Open the file in binary read mode
import pickle
with open('scores_FlanT5_with_AC_PI.pkl', 'rb') as file:
    # Load the array from the file
    scores = pickle.load(file)